<a href="https://colab.research.google.com/github/andrewshandy/ML_PyTorch_Implementations/blob/main/DCGAN_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google_drive_downloader import GoogleDriveDownloader as gdd
dl = "18UTENzuzvwViI0c9uELD4eN4Z9W0H83b"
gdd.download_file_from_google_drive(file_id=dl,
                                    dest_path='./img_align_celeba.zip',
                                    unzip=True)
!mkdir "tmp"
! cp -R "/content/img_align_celeba" "/content/tmp/"

Mounted at /content/drive
Unzipping...Done.


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda:0')  # не забудьте включить GPU

image_size = 64
batch_size = 64
latent_size = 100

transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # Normalize здесь приводит значения в промежуток [-1, 1]
])

dataset = datasets.ImageFolder(root="/content/tmp",
                              transform=transform)

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def plot_img(img):
    plt.figure(figsize=(2.5,2.5))
    grid = utils.make_grid(img, padding=2, normalize=True).cpu()
    plt.imshow(np.transpose(grid, (1,2,0)))


    plt.show()

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_channels, base_size):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(num_channels, base_size, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(base_size, base_size * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(base_size * 2, base_size * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 4),
            
            nn.Conv2d(base_size * 4, base_size * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 8),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(base_size * 8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),
        )
    def forward(self,x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self, latent_size, num_channels, base_size):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            nn.ConvTranspose2d(latent_size, base_size * 16, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(base_size * 16 ),
            nn.LeakyReLU(0.2),

            # (base_size*16) x 4 x 4
            nn.ConvTranspose2d(base_size * 16, base_size * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 8),
            nn.LeakyReLU(0.2),
            
            # (base_size*4) x 8 x 8
            nn.ConvTranspose2d(base_size * 8, base_size * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 4),
            nn.LeakyReLU(0.2),

            # (base_size*2) x 16 x 16
            nn.ConvTranspose2d(base_size * 4, base_size * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(base_size * 2),
            nn.LeakyReLU(0.2),

            # (base_size) x 32 x 32
            nn.ConvTranspose2d(base_size * 2, num_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
num_epochs = 10
learning_rate = 0.0002

G = Generator(100, 3, image_size).to(device)
D = Discriminator(3, image_size).to(device)

img_list = []
G_losses = []
D_losses = []
iters = 0

optim_G = torch.optim.Adam(G.parameters(), lr=learning_rate)
optim_D = torch.optim.Adam(D.parameters(), lr=learning_rate)

fixed_noise = torch.randn(1, latent_size, 1, 1).to(device)
criterion = nn.BCELoss()


for epoch in range(num_epochs):
    for (data, _) in loader:

        data = data.to(device)
        batch_size = data.shape[0]
        # 2 Discriminator
        D.zero_grad()
        
        label = torch.FloatTensor(batch_size).uniform_(0.7, 1.2).to(device)
        output = D(data).reshape(-1)
        
        D_loss_real = criterion(output, label)

        noise = torch.randn(batch_size, latent_size, 1, 1).to(device)
        fake = G(noise)
        label = torch.FloatTensor(batch_size).uniform_(0.0, 0.3).to(device)

        output = D(fake.detach()).reshape(-1)
        D_loss_fake = criterion(output, label)
        D_loss = (D_loss_real + D_loss_fake)/2
        D_loss.backward()
        optim_D.step()


        # 2 Generator 

        G.zero_grad()
        label = torch.FloatTensor(batch_size).uniform_(0.7, 1.2).to(device)
        output = D(fake).reshape(-1)
        
        G_loss = criterion(output, label)
        G_loss.backward()
        optim_G.step()

        

        if iters % 50 == 0:

            print(f'{epoch}/{num_epochs}, {iters/len(loader)}')
            print(f'  G loss: {G_loss}')
            print(f'  D loss: {D_loss}')
            print()
            
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())

        if iters % 50 == 0:
            print("current image")
            img = G(fixed_noise)
            plot_img(img)

        if iters % 50 == 0:
            torch.save(D.state_dict(), '/content/drive/My Drive/D.pt')
            torch.save(G.state_dict(), '/content/drive/My Drive/G.pt')
        iters += 1
        
        if iters % 2500 == 0:
            img = G(fixed_noise)
            img_list.append(img)